# Training notebook

In [ ]:
import os  # for file operations
import json  # for loading the annotations file
from PIL import Image, ImageDraw, ImageOps, ImageEnhance  # for processing the image data
import numpy as np
from random import shuffle
import nvidia.cudnn
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm.keras import TqdmCallback

## constants

In [ ]:
DATA_PATH = "./data/"  # must contain multiple subdirectories - one for each class
ANNOTATIONS_PATH = "./via-annotations.json"  # relative to the notebook

BATCH_SIZE = 32
IMAGE_RESOLUTION = (224, 224)  # standard for VGG19
EPOCHS = 80

CLASS_NAMES = sorted([os.path.basename(f) for f in os.scandir(DATA_PATH) if f.is_dir()])  # automatically generated
CLASS_COUNT = len(CLASS_NAMES)

In [ ]:
with open(ANNOTATIONS_PATH, "r") as via_data:
    via_annotations = json.load(via_data)

In [ ]:
print(CLASS_NAMES)

## segmentation

In [ ]:
annotations = []

num_images = len(via_annotations["_via_img_metadata"])
from tqdm.notebook import tqdm  # the progress bar
import random
for i, image_data in tqdm(enumerate(via_annotations["_via_img_metadata"].items()), total=num_images):
    image_info = image_data[1]
    image_id = image_data[0]
    filename = os.path.join(DATA_PATH, image_info["filename"])

    # Load the image
    try:
        img = Image.open(filename)
        # Rotate JPEGs according to the data provided by the camera
        img = ImageOps.exif_transpose(img)
    except:
        print(f"File not found: {filename}. Skipping.")
        continue
    
    WIDTH, HEIGHT = img.size  # original ones
    img = img.resize(IMAGE_RESOLUTION, resample=Image.NEAREST)

    # Create the region mask (1 means transparent)
    if image_info["regions"]:
        mask = Image.new("1", (WIDTH, HEIGHT))
    else:
        # If there aren't any regions, keep the whole image for now
        mask = Image.new("1", (WIDTH, HEIGHT), "white")
    black = Image.new("RGB", IMAGE_RESOLUTION, (0, 0, 0))

    # Extract regions (polygons, circles, rectangles, ellipses) from VIA annotations and draw them on the mask
    for region_info in image_info["regions"]:
        shape_attributes = region_info["shape_attributes"]
        region_shape = shape_attributes["name"]

        pencil = ImageDraw.Draw(mask)

        if region_shape == "polygon":
            points = [(x, y) for x, y in zip(shape_attributes["all_points_x"], shape_attributes["all_points_y"])]
            pencil.polygon(points, "white")

        elif region_shape == "circle":
            x = shape_attributes["cx"]
            y = shape_attributes["cy"]
            r = shape_attributes["r"]
            points = [(x-r, y-r), (x+r, y+r)]
            pencil.ellipse(points, "white")

        elif region_shape == "ellipse":
            ellipse = Image.new("RGBA", IMAGE_RESOLUTION, (0, 0, 0, 0))
            pencil_ellipse = ImageDraw.Draw(ellipse)
            x = shape_attributes["cx"]
            y = shape_attributes["cy"]
            rx = shape_attributes["rx"]
            ry = shape_attributes["ry"]
            angle = shape_attributes["theta"]
            
            # Since ellipses can't be easily rotated in PIL, I'll have to paste a rotated version on top of the mask
            points = [(x-rx, y-ry), (x+rx, y+ry)]
            pencil_ellipse.ellipse(points, "white")
            ellipse = ellipse.rotate(angle, expand=False, center=(x, y))
            mask.paste(ellipse, (0, 0))

        elif region_shape == "rect":
            x = shape_attributes["x"]
            y = shape_attributes["y"]
            w = shape_attributes["width"]
            h = shape_attributes["height"]

            points = [(x, y), (x+w, y+h)]
            pencil.rectangle(points, "white")
    
    # Resize the mask as well
    mask = mask.resize(IMAGE_RESOLUTION, resample=Image.NEAREST).convert("1")
    masked = Image.composite(img, black, mask)

    # Plot the three images
    import matplotlib.pyplot as plt
    f, axarr = plt.subplots(1, 3)
    plt.axis("on")
    axarr[0].imshow(img)
    axarr[0].set_title("Original")
    
    axarr[1].imshow(masked)
    axarr[1].set_title("Combined")

    axarr[2].imshow(mask, cmap="copper")
    axarr[2].set_title("Mask")

    plt.show()
    
    # Get the class label from the directory structure
    class_label = os.path.basename(os.path.dirname(filename))

    # Convert class label to one-hot encoding
    class_index = CLASS_NAMES.index(class_label)
    class_one_hot = to_categorical(class_index, num_classes=CLASS_COUNT)
    
    # Preprocess
    n_copies = random.randint(1, 6)
    f, axarr = plt.subplots(1, n_copies + 1)
    plt.axis("off")
    for j in range(0, n_copies):
        random.seed(i+j // random.randint(1, 4) + random.randint(0, 16777216))
        
        preprocessed = masked.rotate(random.randint(0, 3) * 90 + random.randint(-22, 22))
        sharpness = ImageEnhance.Sharpness(img)
        color = ImageEnhance.Color(img)
        contrast = ImageEnhance.Contrast(img)
        brightness = ImageEnhance.Brightness(img)
        
        sharpness.enhance(random.uniform(0.5, 1.5))
        color.enhance(random.uniform(0.75, 1.25))
        contrast.enhance(random.uniform(0.625, 1.375))
        contrast.enhance(random.uniform(0.5, 1.5))

        annotations.append((np.asarray(preprocessed) / 255.0, class_one_hot))
        axarr[j].imshow(preprocessed)

# Shuffle the data
shuffle(annotations)

# Convert them to Numpy arrays
images = np.array([i[0] for i in annotations])
labels = np.array([i[1] for i in annotations])


## model definition

In [ ]:
base_model = VGG19(weights="imagenet", include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(CLASS_COUNT, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


## compilation

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0002),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

## training

In [ ]:
history = model.fit(
    images,
    labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=0,
    callbacks=[TqdmCallback()],
    validation_split=0.2
)

In [ ]:
model.save("model.h5")

### fine-tuning

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("model.h5")

In [ ]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    images,
    labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=0,
    callbacks=[TqdmCallback()],
    validation_split=0.2
)

In [ ]:
model.save("model.h5")

## inferencing

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model = load_model("model.h5", compile=False)

In [ ]:
input_image = load_img("../tester.jpg", target_size=(224, 224))
input_image = img_to_array(input_image)
input_image = input_image / 255.0
input_image = input_image.reshape((1,) + input_image.shape)

In [ ]:
predictions = model.predict(input_image)
predicted_class = np.argmax(predictions, axis=1)[0]
predicted_label = CLASS_NAMES[predicted_class]

## plot

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(input_image[0])
plt.axis("off")
plt.title(predicted_label, fontsize=18)
plt.show()